In [1]:
import sys
sys.path.insert(0, '.')
from utils import *

# Some simple forecasting methods

We are going to start with some really simple models to predict time series.


In [3]:
d = pd.read_csv('data/aus_production.csv')
aus_production = (
    d
    .assign(Date=pd.to_datetime(d.Quarter.str.replace(' ', '')))
    .pipe(compute, lambda x: dict(Year=x.Date.dt.year))
    .set_index('Date', drop=False)
    .drop(columns='Date Year'.split())
)


/var/folders/72/x6_kg98117nd5xkqbx_bl3dc0000gn/T/ipykernel_41835/1257075650.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  .assign(Date=pd.to_datetime(d.Quarter.str.replace(' ', '')))


In [ ]:
bricks = aus_production['1970-01-01':'2004-01-01']

In [ ]:
class SimpleTSModel:
    """Simple TS model base class."""
    def __init__(self, y):
        """Determine y data and sampling frequency."""
        if not hasattr(y, 'index'):
            y = pd.Series(y)
        if hasattr(y.index, 'inferred_freq'):
            self.y = y.asfreq(y.index.inferred_freq)
            self.freq = self.y.index.freq
        else:
            self.y = y.copy()
            self.freq = None

    def fit(self):
        """Nothing to do here, but in other libraries this is a method that does things."""
        return self

    def forecast(self, dt=None, end=None, periods=None):
        """Make a forward-looking prediction."""
        assert sum([dt is None, end is None, periods is None]) == 2
        tmax = self.y.index.max()
        if dt is not None:
            end = tmax + (pd.to_timedelta(dt) if self.freq else dt)
        elif end is not None:
            end = pd.to_datetime(end) if self.freq else end
        elif periods is not None:
            end = tmax + periods * (self.freq or 1)
        return self.predict(tmax + 1 * (self.freq or 1), end)

    def _normalize_times(self, start, end):
        """Do some tedious datetime manipulation."""
        Y = self.y
        t0 = Y.index.min()
        if start is None:
            start = t0
        if end is None:
            end = Y.index.max()
        if self.freq is not None:
            start = pd.to_datetime(start)
            end = pd.to_datetime(end)
        if self.freq:
            index = pd.date_range(t0, end, freq=self.freq)
        else:
            index = np.arange(t0, end+1)
        return start, end, index

class TSMean(SimpleTSModel):
    """The future will look like the average of the past."""
    def predict(self, start=None, end=None):
        # value is always the mean
        Y = self.y
        start, end, index = self._normalize_times(start, end)
        m = Y.mean()
        out = pd.Series(m, index=index)
        out = out.loc[start:].copy()
        return out

class TSNaive(SimpleTSModel):
    """Tomorrow will look like today."""
    def predict(self, start=None, end=None):
        # tomorrow probably same as today
        Y = self.y
        start, end, index = self._normalize_times(start, end)
        out = pd.Series(np.nan, index=index)
        out.loc[:Y.index.max()] = Y
        out = out.shift(1)
        out.loc[Y.index.max():] = Y.iloc[-1]
        out = out.loc[start:].copy()
        return out.copy()

class TSNaiveSeasonal(SimpleTSModel):
    """Next year will fluctuate the same way as this year."""
    def __init__(self, y, lag):
        super(TSNaiveSeasonal, self).__init__(y)
        self.lag = lag
        assert self.y.index.min() + lag * (self.freq or 1) < self.y.index.max(), \
            'lag must be less than input timeseries'

    def predict(self, start=None, end=None):
        # tomorrow probably same as this time last year/month/whatever
        Y, lag = self.y, self.lag
        start, end, index = self._normalize_times(start, end)
        out = pd.Series(np.nan, index=index)
        out.loc[:Y.index.max()] = Y
        out = out.shift(lag)
        i = 0
        while np.isnan(out.iloc[-1]):
            mask = out.isna()
            out[mask] = out.shift(lag)[mask]
            i += 1
        out = out.loc[start:].copy()
        return out.copy()

class TSDrift(SimpleTSModel):
    """Draw a line from t=0 thru today, and extrapolate to tomorrow."""
    def predict(self, start=None, end=None):
        # value extrapolated based on slope wrt first observation
        # TODO: might be slightly wrong
        # doesn't *quite* agree with R's RW(Y~drift()) ?
        Y = self.y
        Y0 = Y.values[0]
        YT = Y.shift(-1)
        start, end, index = self._normalize_times(start, end)
        YT = pd.Series(np.nan, index=index)
        YT.loc[Y.index.min():Y.index.max()] = Y
        YT = YT.shift(1)
        YT.iloc[0] = Y.iloc[0]
        h = pd.Series(1, index=index)
        extrap_mask = YT.isna()
        h.loc[YT.isna()] = np.arange(1, extrap_mask.sum()+1)
        YT.loc[extrap_mask] = Y.iloc[-1]
        x = np.maximum(1, np.arange(len(YT)) - 1)
        out = YT + h * ((YT - Y0) / x)
        out.iloc[0] = np.nan
        out = out.loc[start:].copy()
        return out.copy()

## Mean Model
* Just predict future as the mean of past values

In [ ]:
m = TSMean(bricks.Bricks)

In [ ]:
fig, ax = plt.subplots()
ax.plot(bricks.Bricks, color='k')
ax.plot(m.forecast(end='2010'), color='C2', label='mean')
ax.legend(**legend_right)
ax.grid()

## Naive

* Simply set all forecasts to be the value of the last observation
$$\hat{y}_{T+h|T} = y_{T}.$$


In [ ]:
n = TSNaive(bricks.Bricks).fit()

In [ ]:
fig, ax = plt.subplots()
ax.plot(bricks.Bricks, color='k')
ax.plot(n.forecast(end='2010'), color='C0', label='naïve')
ax.legend(**legend_right)
ax.grid()

## Seasonal naive method
* Each forecast will be equal to the last observed value from the same season
$$\hat{y}_{T+h|T} = y_{T+h-m(k+1)},$$

In [ ]:
s = TSNaiveSeasonal(bricks.Bricks, 4)

In [ ]:
fig, ax = plt.subplots()
ax.plot(bricks.Bricks, color='k')
ax.plot(s.forecast(end='2010'), color='C1', label='seasonal naïve')
ax.legend(**legend_right)
ax.grid()

## Drift method
* Method to capture the increase or decrease over time
* Equivalent to drwaing a line between the first and last observations, and extrapolating it into the future

$$ \hat{y}_{T+h|T} = y_{T} + \frac{h}{T-1}\sum_{t=2}^T (y_{t}-y_{t-1}) = y_{T} + h \left( \frac{y_{T} -y_{1}}{T-1}\right).$$

In [ ]:
dr = TSDrift(bricks.Bricks)

In [ ]:
fig, ax = plt.subplots()
ax.plot(bricks.Bricks, color='k')
ax.plot(dr.forecast(end='2010'), color='C3', label='drift')
ax.legend(**legend_right)
ax.grid()

In [ ]:
fig, ax = plt.subplots()
ax.plot(bricks.Bricks, color='k')
ax.plot(n.forecast(end='2010'), color='C0', label='naïve')
ax.plot(s.forecast(end='2010'), color='C1', label='seasonal naïve')
ax.plot(m.forecast(end='2010'), color='C2', label='mean')
ax.plot(dr.forecast(end='2010'), color='C3', label='drift')
ax.legend(**legend_right)
ax.grid()

## Another example: Australian quarterly beer production

In [ ]:
Y = aus_production.Beer
Ytrain = Y[:'2006']
m = TSMean(Ytrain)
n = TSNaive(Ytrain).fit()
s = TSNaiveSeasonal(Ytrain, 4)
dr = TSDrift(Ytrain)

In [ ]:
fig, ax = plt.subplots()
ax.plot(Ytrain, 'k')
ax.plot(Y, 'k:')
ax.plot(n.forecast(end='2010'), color='C0', label='naïve')
ax.plot(s.forecast(end='2010'), color='C1', label='seasonal naïve')
ax.plot(m.forecast(end='2010'), color='C2', label='mean')
ax.plot(dr.forecast(end='2010'), color='C3', label='drift')
ax.legend(**legend_right)
ax.grid()

## Fitted values
* How well the method fits the data
* How well the mothod forecasts

* $\hat{y}_{t|t-1}$ is the forecast of $y_t$ based on observations $y_{1},\dots,y_{t-1}$
* We call these "fitted values"
* Sometimes drop the subscript: $\hat{y}_{t} = \hat{y}_{t|t-1}$
* $\hat{y}_{t} = \overline{y}_t$ for average method
* $\hat{y}_{t} = y_{t-1} + (y_t - y_1)/(T-1)$ for drift method

## Residuals

* difference between observed value and its fitted value
* Assumptions:
    * $\{e_t\}$ uncorrelated. If they aren't, then information left in residuals that should be used in computing forecasts
    * $\{e_t\}$ have mean zero. If they don't, then forecasts are biased
* Useful properties (for distributions & prediction intervals
    * $\{e_t\}$ have constant variance
    * $\{e_t\}$ are normally distributed

In [ ]:
GOOG = (
    pd.read_csv('data/gafa_stock.csv')
    .query("Symbol == 'GOOG'")
    .sort_values('Date')
    .reset_index(drop=True)
    .pipe(compute, lambda x: dict(Date = pd.to_datetime(x.Date, format='%Y-%m-%d')))
)

In [ ]:
GOOG_2015 = GOOG.query('Date.dt.year == 2015')

In [ ]:
len(GOOG_2015)

In [ ]:
Y = GOOG.Close
Ytrain = GOOG_2015.Close
nsamples = len(GOOG_2015)
m = TSMean(Ytrain)
n = TSNaive(Ytrain).fit()
s = TSNaiveSeasonal(Ytrain, nsamples - 2)
dr = TSDrift(Ytrain)

In [ ]:
m.predict()

In [ ]:
fig, ax = plt.subplots()
ax.plot(Y[GOOG.Date.between('2015-01-01', '2016-01-30')], color='k')
c = 'C3 C2 C0 C4'.split()
ax.plot(m.predict(),  ls='--', color=c[0])
ax.plot(n.predict(),  ls='--', color=c[1])
ax.plot(s.predict(),  ls='--', color=c[2])
ax.plot(dr.predict(), ls='--', color=c[3])
ax.plot(m.forecast(30),  color=c[0], label='Mean')
ax.plot(n.forecast(30),  color=c[1], label='Naive')
ax.plot(s.forecast(30),  color=c[2], label='Seasonal Naive')
ax.plot(dr.forecast(30), color=c[3], label='Drift')
ax.set(ylabel='Closing Price (USD)')
ax.legend(loc='center left', bbox_to_anchor=[1, .5])
suptitle('Google stock')
ax.set(title='daily close thru Jan 2016')
ax.grid()

In [ ]:
results = GOOG_2015.assign(
    mean=m.predict(),
    naive=n.predict(),
    naive_seasonal=s.predict(),
    drift=dr.predict(),
    resid_mean=m.predict() - Ytrain,
    resid_naive=n.predict() - Ytrain,
    resid_naive_seasonal=s.predict() - Ytrain,
    resid_drift=dr.predict() - Ytrain,
)
results.head()


In [ ]:
def plot_tsresiduals(Y, y, acf_lags=np.r_[1:26]):
    """Plot timeseries residuals for ground truth Y and estimate y."""
    fig = plt.figure()
    gs = plt.GridSpec(3, 2, figure=fig)
    ts_ax = fig.add_subplot(gs[0, :])
    axs = np.array([ts_ax] + [fig.add_subplot(gs[i, j]) for j in (0, 1) for i in (1, 2)])
    ax, rax, hax, acfax, pacfax = axs
    mask = ~(np.isnan(Y) | np.isnan(y))
    Y, y = Y[mask], y[mask]
    dy = Y - y
    ax.plot(Y, color='k')
    ax.plot(y)
    ax.set(title='Time Series')
    lim = 1.1 * max(-dy.min(), dy.max())
    lim = -lim, lim
    rax.plot(dy)
    rax.set(ylim=lim, title='Residuals')
    sns.histplot(dy, bins=np.linspace(lim[0], lim[1], 22),
             kde=True, stat='density', ax=hax)
    hax.set(title='Residual Distribution')
    sm.graphics.tsa.plot_acf(dy, lags=acf_lags, ax=acfax)
    acfax.set_ylim(-0.5, 0.5)
    sm.graphics.tsa.plot_pacf(dy, lags=acf_lags, ax=pacfax)
    pacfax.set_ylim(-0.5, 0.5)
    for a in axs.ravel():
        a.grid()
    plt.tight_layout()
    return fig, axs


In [ ]:
plot_tsresiduals(Ytrain, results.naive)
suptitle('Naïve forecast')
plt.subplots_adjust(top=.9)

In [ ]:
plot_tsresiduals(Ytrain, results.drift)
suptitle('Naïve forecast')
plt.subplots_adjust(top=.9)

## Forecast Distributions

* All forecasts have an uncertainty
* We express the uncertainty in our forecasts using a probability distribution
* Describes the probability of observing possible future values using the fitted model
* Most time series models produce normally distributed forecasts

## Prediction intervals


In [ ]:
mult = pd.DataFrame(dict(Percentage=np.r_[50:90:5, 90:100]))
mult['Multiplier'] = stats.norm.isf((1 - mult.Percentage/100) / 2)
mult = mult.set_index('Percentage')
mult

In [ ]:
fig, ax = plt.subplots()
g = GOOG[GOOG.Date.lt('2016-02-01')]
ax.plot(GOOG_2015.Close, color='.3', zorder=-10, label='data')
fc = n.forecast(10)
ax.plot(fc, lw=1.5, label='naïve forecast')
sigma = results.resid_naive.std()
m80, m95 = mult.Multiplier.loc[[80, 95]]
didx = fc.index - fc.index.min()
ax.fill_between(fc.index, fc - m80*sigma * np.sqrt(didx), fc + m80*sigma * np.sqrt(didx),
                alpha=.33, lw=0, label='80%')
ax.fill_between(fc.index, fc - m95*sigma * np.sqrt(didx), fc + m95*sigma * np.sqrt(didx),
                alpha=.25, color='C0', lw=0, label='95%')
ax.legend(loc='center left', bbox_to_anchor=[1, .5])
suptitle('Google stock')
ax.set(title='daily close thru Jan 2016', ylabel='Closing Price (USD)')
ax.grid()

## Forecasting with decomposition

In [ ]:
d = pd.read_csv('data/us_employment.csv')
d = us_retail_employment = (
    d
    .assign(date=pd.to_datetime(d.Month, format='%Y %b'))
    .pipe(compute, lambda x: dict(year=x.date.dt.year))
    .query("year >= 1990 and Title == 'Retail Trade'")
    .set_index('date')
    .drop(columns='year Series_ID'.split())
)
us_retail_employment

In [ ]:
stl = sm.tsa.STL(d.Employed).fit()

In [ ]:
s_stl = TSNaiveSeasonal(stl.seasonal, 12)

In [ ]:
dr_stl = TSDrift(d.Employed - stl.seasonal)


In [ ]:
from datetime import timedelta
end = us_retail_employment.index.max() + + timedelta(days=365*4)

In [ ]:
fig, ax = plt.subplots(figsize=sizets)
ax.plot(d.Employed, 'k')
end = pd.to_datetime('2025-01-01')
ax.plot(s_stl.predict(end=end) + dr_stl.predict(end=end))
ax.plot(s_stl.forecast(end=end) + dr_stl.forecast(end=end))
ax.set(ylabel='Employed')
ax.grid()

In [ ]:
plot_tsresiduals(d.Employed, s_stl.predict() + dr_stl.predict());

## Evaluating forecast accuracy

In [ ]:
aus_production['1995':]

In [ ]:
aus_production[aus_production.index.quarter == 1]

In [ ]:
aus_production.iloc[-20:]

In [ ]:
recent_production = aus_production['1992':]
split = '2007-12-01'
beer_train = recent_production[:split]
beer_test = recent_production[split:]

Ytrain = beer_train.Beer
ms = dict(
    Drift = TSDrift(Ytrain),
    Mean = TSMean(Ytrain),
    Naive = TSNaive(Ytrain),
    SeasonalNaive = TSNaiveSeasonal(Ytrain, 4),
)

In [ ]:
fig, ax = plt.subplots()
Y = recent_production.Beer
ax.plot(Y, color='k')
c = 'C3 C2 C0 C4'.split()
for ((label, model), c) in zip(ms.items(), c):
    ax.plot(model.predict(),  ls=':', color=c, alpha=.5)
    ax.plot(model.forecast(end=Y.index.max()),  color=c, label=label)
ax.set(ylabel='Megalitres')
ax.legend(loc='center left', bbox_to_anchor=[1, .5])
suptitle('Google stock')
ax.set(title='Forecasts for quarterly beer production')
ax.grid()


In [ ]:
def MAE(Y, y):
    """Mean absolute error."""
    return np.mean(np.abs(Y - y))


def MAPE(Y, y):
    """Mean absolute percent error."""
    return 100 * np.mean(np.abs((Y - y) / Y))




def tsaccuracy(Ytest, models):
    """Gather some metrics for a few models."""
    fs = RMSE, MAE, MAPE
    return pd.DataFrame({
        label: [f(Ytest, model.predict(Ytest.index.min(), Ytest.index.max()))
                for f in (RMSE, MAE, MAPE)]
        for (label, model) in models.items()
    }, index=[f.__name__ for f in fs]).T

In [ ]:
tsaccuracy(beer_test.Beer, ms)

In [ ]:
results.naive